<a href="https://colab.research.google.com/github/shangeth/wavencoder/blob/master/examples/notebooks/wavencoder_demo_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Demo Notebook
## [wavencoder](https://pypi.org/project/wavencoder/) Models

---


Author : Shangeth Rajaa

![Twitter Follow](https://img.shields.io/twitter/follow/shangethr?style=social)

[GitHub](https://github.com/shangeth) [LinkedIn](https://www.linkedin.com/in/shangeth/)

# Installing wavencoder

In [ ]:
!pip install fairseq
!pip install wavencoder

     |████████████████████████████████| 307kB 4.7MB/s 
     |████████████████████████████████| 71kB 7.4MB/s 
  Created wheel for fairseq: filename=fairseq-0.9.0-cp36-cp36m-linux_x86_64.whl size=2046423 sha256=f68a8ad6a5ce93540273cc72dd2d81ba74683c60a9aeb5aad1b93c187a2bb6e5
  Stored in directory: /root/.cache/pip/wheels/37/3e/1b/0fa30695dcba41e4b0088067fa40f3328d1e8ee78c22cd4766
Successfully built fairseq


# Wav2Vec pretrained feature extractor

In [ ]:
import torch
import wavencoder

x = torch.randn(1, 16000) # [1, 16000]
encoder = wavencoder.models.Wav2Vec(pretrained=True)
z = encoder(x) # [1, 512, 99]
z.shape

wav2vec_large.pt: 325MB [00:07, 45.7MB/s]                           


torch.Size([1, 512, 98])

# SincNet pretrained feature extractor

In [ ]:
from wavencoder.models import SincNet
encoder = SincNet(pretrained=True).eval()
x = torch.randn(1, 3200) 
z = encoder(x)
print(z.shape)

torch.Size([1, 6420])


# Audio Classifier
- wav2vec encoder `[1, 16000] -> [1, 512, 98]`
- mean of features along time axis `[1, 512, 98] -> [1, 512]`
- ANN Classifier `[1, 512] -> [1, 2]`


In [ ]:
import torch
import torch.nn as nn
import wavencoder

class AudioClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = wavencoder.models.Wav2Vec(pretrained=True)
        self.classifier = nn.Linear(512, 2)

    def forward(self, x):
        z = self.encoder(x)
        z = torch.mean(z, dim=2)
        out = self.classifier(z)
        return out

model = AudioClassifier()
x = torch.randn(1, 16000)
y_hat = model(x)
print(y_hat.shape)

torch.Size([1, 2])


- SincNet encoder `[1, 3200] -> [1, 6420]`
- ANN Classifier `[1, 6420] -> [1, 512]`

In [ ]:
import torch
import torch.nn as nn
import wavencoder

class SincNetAudioClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = SincNet(pretrained=True)
        self.classifier = nn.Linear(6420, 2)

    def forward(self, x):
        z = self.encoder(x)
        out = self.classifier(z)
        return out

model = SincNetAudioClassifier()
x = torch.randn(2, 3200)
y_hat = model(x)
print(y_hat.shape)

torch.Size([2, 2])


# LSTM Attention Classifier

In [ ]:
import torch
import torch.nn as nn
import wavencoder

model = nn.Sequential(
        wavencoder.models.Wav2Vec(),
        wavencoder.models.LSTM_Attn_Classifier(512, 64, 2)
)

x = torch.randn(1, 16000)
y_hat, attn_weights = model(x)

print(y_hat.shape, attn_weights.shape)

torch.Size([1, 2]) torch.Size([1, 98])
